In [35]:
# Relax a matroid
# This is honestly terrible code; it switches between containers a billion times
def relax_circuit_hyperplane(M):

    # Get circuits and hyperplanes
    circuits = [tuple(c) for c in sorted([sorted(C) for C in M.circuits()])]
    hyperplanes = [tuple(h) for h in sorted([sorted(F) for F in M.hyperplanes()])]
    bases = [sorted (B) for B in M.bases()]

    # Find circuit-hyperplanes
    circuit_hyperplanes = list(set(circuits).intersection(hyperplanes))

    # Return if nothing to relax
    if not circuit_hyperplanes:
        print('No circuit hyperplanes to relax!')
        return None
    
    # Print circuit-hyperplanes
    for ch in enumerate(circuit_hyperplanes):
        print('{0}: '.format(ch[0]) + str(ch[1]))

    # Get circuit-hyperplane to relax
    ch = list(circuit_hyperplanes[int(input())])
    
    #print(bases)

    # Add it as a basis and create new list
    bases.append(ch)
    #new_bases = [''.join(l) for l in bases]
    
    return Matroid(bases=bases)

# cdgh is the circuit-hyperplane to relax to get Vamos from NonVamos
# NV = matroids.named_matroids.NonVamos()
# V = relax_circuit_hyperplane(NV)
# if V.is_isomorphic(matroids.named_matroids.Vamos()):
#    print('NonVamos successfully relaxed to Vamos')

# Get Pappus
# P = matroids.named_matroids.Pappus()

# Plot Pappus to find to relax
# pos = {'a':(0,2), 'b': (1,2), 'c':(2,2), 'd':(0,0), 'e':(1,0), 'f':(2,0), 'g':(.5,1),'h':(1,1), 'i':(1.5,1)}
# P.plot(pos_method=1, pos_dict=pos)

# Relax Pappus and check
# NP = relax_circuit_hyperplane(P)
# if NP.is_isomorphic(matroids.named_matroids.NonPappus()):
#    print('Pappus successfully relaxed to NonPappus')

In [24]:
# MAX CODE FOR TZF
# given a lattice L and a flag F in L containing at least the min and max
# of L, xmf(L,F) is the product of reduced characteristic polynomials of
# intevals in F, and xmf1(L,F) finds the value of this polynomial when you set
# q = 1.

def xmf(L,F):
    l = len(F)
    R.<q> = QQ['q']
    numerator = product([posetify(L.interval(F[i],F[i+1])).characteristic_polynomial() for i in [0..(l-2)]])
    denominator = (q - 1)^(l-1)
    return (numerator/denominator)

def xmf1(L,F):
    l = len(F)
    R.<q> = QQ['q']
    numerator = product([posetify(L.interval(F[i],F[i+1])).characteristic_polynomial() for i in [0..(l-2)]])
    denominator = (q - 1)^(l-1)
    return (numerator/denominator)(1)

# given a lattice L and a flag F in L containing at least the min and
# max of L, trf(L,F) finds the rational function (in the variable s)
# that you need when building the topological zeta function.
def trf(L,F):
    R.<s> = QQ['s']
    rk = L.rank_function()
    return product([1/((len(f))*s + rk(f)) for f in F if not f == L.bottom()])
    
# Utility: For a set S of subsets, form its associated poset w.r.t. containment
def posetify(S):
    return Poset((S,[[a,b] for a in S for b in S if a.issubset(b)]))

def tzf(M):
    L = M.lattice_of_flats()
    flags = [c for c in L.chains() if len(c) > 0 and c[0] == L.bottom() and c[-1] == L.top()]
    return sum([xmf1(L,f)*trf(L,f) for f in flags])

In [169]:
def isets_zeta(M, zeta=False):

    # Create relevant uniform matroid
    unif = matroids.Uniform(M.rank(), M.size())
    
    # Print relevant information
    print(M)
    print([len(M.dependent_r_sets(r)) for r in range(0, M.rank()+1)])
        
    print(unif)
    print([len(unif.dependent_r_sets(r)) for r in range(0, M.rank()+1)])
    
    if zeta:
        print('TZF for M: ', tzf(M))

In [189]:
g = Matroid(Graph([(0,1,1), (0,1,2), (0,1,3), (1,2,1), (2,3,1)], multiedges=True))
isets_zeta(g, True)

Graphic matroid of rank 3 on 5 elements
[0, 0, 3, 7]
U(3, 5): Matroid of rank 3 on 5 elements with circuit-closures
{3: {{0, 1, 2, 3, 4}}}
[0, 0, 0, 0]
TZF for M:  1/3/(s^3 + 7/3*s^2 + 5/3*s + 1/3)


In [212]:
g = Matroid(Graph([(0,1,1), (1,2,1), (1,2,2), (0,1,2), (1,2,3)], multiedges=True))


matroids.Wheel(5).lattice_of_flats().characteristic_polynomial()
#print(g.lattice_of_flats().characteristic_polynomial())

q^5 - 10*q^4 + 40*q^3 - 80*q^2 + 79*q - 30

In [205]:
tzf(matroids.Wheel(4))

(-3/8*s^6 - 1/12*s^5 + 5/12*s^4 - 1/5*s^3 - 4/15*s^2 + 9/40*s + 3/20)/(s^7 + 331/60*s^6 + 1549/120*s^5 + 83/5*s^4 + 38/3*s^3 + 86/15*s^2 + 57/40*s + 3/20)

In [213]:
tzf(Matroid(g))

1/6/(s^2 + 5/6*s + 1/6)